In [ ]:
pip install delta-spark

In [ ]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

In [ ]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+---------+
|namespace|
+---------+
|  default|
|     demo|
|      nba|
+---------+



In [9]:
spark.sql(
    """
    SHOW TABLES FROM nba
    """
).show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|     nba|bz_players|      false|
|     nba|     teams|      false|
+--------+----------+-----------+



In [11]:
spark.sql(
    """
    DROP TABLE IF EXISTS nba.bz_abrev
    """
)

DataFrame[]

In [12]:
spark.sql(
    """
    CREATE EXTERNAL TABLE nba.bz_abrev (
        abreviation STRING,
        name_team STRING,
        league STRING
    )
    USING DELTA
    TBLPROPERTIES (
   'skip.header.line.count'='1'
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/bz_abrev'
    """
)

21/11/30 00:15:16 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`bz_abrev` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
21/11/30 00:15:16 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [13]:
spark.sql(
    """
    SHOW TABLES FROM nba
    """
).show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|     nba|  bz_abrev|      false|
|     nba|bz_players|      false|
+--------+----------+-----------+



In [14]:
spark.sql(
    """
    SELECT * FROM nba.bz_abrev
    """
).show()

+-----------+---------+------+
|abreviation|name_team|league|
+-----------+---------+------+
+-----------+---------+------+



In [15]:
#read hdfs file to dataframe
#StructField("header", IntegerType(), True),
from pyspark.sql.types import *        
file_location = "hdfs://hdfs-nn:9000/projeto/triplos/bronze/teams_abrev"

df = spark\
    .read\
    .format("csv")\
    .option('header','true')\
    .option("inferSchema" , "true")\
    .option("delimiter",",")\
    .load(file_location)

df.printSchema()
df.show()

root
 |-- abreviation: string (nullable = true)
 |-- name_team: string (nullable = true)
 |-- league: string (nullable = true)

+-----------+--------------------+--------------+
|abreviation|           name_team|        league|
+-----------+--------------------+--------------+
|        AKF|Akron Firestone N...|         (NBL)|
|        AKG|Akron Goodyear Wi...|        (NBL) |
|        ANA|      Anaheim Amigos|         (ABA)|
|        AND|Anderson Duffey P...|(NBL/NBA/NPBL)|
|        ATC|    Atlanta Crackers|       (PBLA) |
|        ATG|       Atlanta Glory|        (ABL) |
|        ATL|       Atlanta Hawks|        (NBA) |
|        BAC|     Baltimore Claws|        (ABA) |
|        BAL|   Baltimore Bullets|        (NBA) |
|        BFB|      Buffalo Bisons|        (NBL) |
|        BFG|     Buffalo Germans|        (ABL) |
|        BIR| Birmingham Skyhawks|       (PBLA) |
|        BKA|  Brooklyn Arcadians|        (ABL) |
|        BKC|    Brooklyn Celtics|        (ABL) |
|        BKV|Brooklyn 

In [16]:
#write df to hive deltalake_table
#.select("*")  reorder columns to match parquet table sequence

df \
    .select("abreviation", "name_team", "league") \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/projeto/nba/bz_abrev")

In [17]:
df.show()
df.printSchema()

+-----------+--------------------+--------------+
|abreviation|           name_team|        league|
+-----------+--------------------+--------------+
|        AKF|Akron Firestone N...|         (NBL)|
|        AKG|Akron Goodyear Wi...|        (NBL) |
|        ANA|      Anaheim Amigos|         (ABA)|
|        AND|Anderson Duffey P...|(NBL/NBA/NPBL)|
|        ATC|    Atlanta Crackers|       (PBLA) |
|        ATG|       Atlanta Glory|        (ABL) |
|        ATL|       Atlanta Hawks|        (NBA) |
|        BAC|     Baltimore Claws|        (ABA) |
|        BAL|   Baltimore Bullets|        (NBA) |
|        BFB|      Buffalo Bisons|        (NBL) |
|        BFG|     Buffalo Germans|        (ABL) |
|        BIR| Birmingham Skyhawks|       (PBLA) |
|        BKA|  Brooklyn Arcadians|        (ABL) |
|        BKC|    Brooklyn Celtics|        (ABL) |
|        BKV|Brooklyn Visitations|        (ABL) |
|        BLB|   Baltimore Bullets|(ABL/BAA/NBA) |
|        BOS|      Boston Celtics|    (BAA/NBA) |


In [19]:
spark.sql(
    """
    DESCRIBE HISTORY nba.bz_abrev
    """
).show()

#isto permite ver o historico de versoes da tabela

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2021-11-30 00:18:...|  null|    null|       WRITE|{mode -> Overwrit...|null|    null|     null|          0|          null|        false|{numFiles -> 1, n...|        null|
|      0|2021-11-30 00:15:...|  null|    null|CREATE TABLE|{isManaged -> fal...|null|    null|     null|       null|          null|         true|                  {}|        null|
+-------+--------------------+------+--------+------------+--------------------+----+--------+------

In [ ]:
spark.stop()